Homework 6

In [ ]:
import pandas as pd
import tqdm
import py2neo

download_url = (
       "https://raw.githubusercontent.com/"
        "master-ds2e/NoSQL/master/data/stackoverflow_network/stack_network_links.csv" )

links = pd.read_csv(download_url)
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="hw6")
graph.delete_all()

download_url2 = (
       "https://raw.githubusercontent.com/"
        "master-ds2e/NoSQL/master/data/stackoverflow_network/stack_network_nodes.csv" )

nodes = pd.read_csv(download_url2)
text = nodes.iterrows()
test = next(text)
print (test[1])


try:
    graph.run("CREATE CONSTRAINT ON (n:Nodes) ASSERT n.name IS UNIQUE")
except:
    print("Constraint already exists")
    
transaction_list = []
transaction = "UNWIND $json as data CREATE (n:Nodes) SET n = data"
iteration = 0
for row in tqdm.tqdm(nodes.iterrows()):
    name = row[1]["name"]
    group = row[1]["group"] 
    nodesize = row[1]["nodesize"] 
    transaction_list.append({"name":name,"group":group, "nodesize":nodesize})
    iteration += 1
    if iteration % 1000 == 0:
        
        graph.run(transaction, json=transaction_list)
        transaction_list = []
graph.run(transaction, json=transaction_list)        
        
transaction_list = []
transaction = "UNWIND $json as data CREATE (n:Links) SET n = data"
iteration = 0
for row in tqdm.tqdm(links.iterrows()):
    source = row[1]["source"]
    target = row[1]["target"] 
    value = row[1]["value"] 
    transaction_list.append({"source":source,"target":target, "value":value})
    iteration += 1
    if iteration % 1000 == 0:
        
        graph.run(transaction, json=transaction_list)
        transaction_list = []
        
graph.run(transaction, json=transaction_list)

Exo 1

In [ ]:
query = """
MATCH (n : Nodes)
WHERE exists(n.nodesize)
RETURN n
ORDER BY n.nodesize DESC
LIMIT 10
"""
data = graph.run(query).to_data_frame()
print(data)

Exo 2

In [ ]:
query2 = """
MATCH (n : Links)
WHERE exists(n.value)
RETURN n
ORDER BY n.value DESC
LIMIT 20
"""
data = graph.run(query2).to_data_frame()
print(data)

Exo 3

In [ ]:
query2 = """
MATCH (n : Nodes)
WHERE n.group = 1
RETURN n.group, n.name
"""
data = graph.run(query2).to_data_frame()
print(data)

Exo 4 : we didn't find the solution   

Exo 5

In [ ]:
# the problem is that we don't have 14 different colors in neo4j

for i in range(1, 15):
    query = """MATCH (n:Nodes)
    WHERE n.group = {}
    SET n : Group{}
    """.format(str(i), (i))
    data = graph.run(query).to_data_frame()
print(data)

Our idea was to create a node for every group and to give each node different color, but in neo4j exists 14 colors and we had 15 groups